# Thanks to https://www.kaggle.com/siavrez/wavenet-keras and Sergey Bryansky

In [1]:
!pip install tensorflow_addons
import tensorflow as tf
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
import random
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa
import gc

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used

In [3]:
# configurations and main hyperparammeters
EPOCHS = 224
NNBATCHSIZE = 16
GROUP_BATCH_SIZE = 4000
SEED = 321
LR = 0.001
SPLITS = 5

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [4]:
# read data
def read_data():
    train = pd.read_csv('/home/lyh/liverpool/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv('/home/lyh/liverpool/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
    sub  = pd.read_csv('/home/lyh/liverpool/sample_submission.csv', dtype={'time': np.float32})
    
    Y_train_proba = np.load("/home/lyh/liverpool/Y_train_proba.npy")
    Y_test_proba = np.load("/home/lyh/liverpool/Y_test_proba.npy")
    
    for i in range(11):
        train[f"proba_{i}"] = Y_train_proba[:, i]
        test[f"proba_{i}"] = Y_test_proba[:, i]
        
    STD = 0.01
    old_data = train['signal']
    new_data = old_data + np.random.normal(0, STD, size=len(train))
    train['signal_noised'] = new_data

    old_data = test['signal']
    new_data = old_data + np.random.normal(0, STD, size=len(test))
    test['signal_noised'] = new_data
        

    return train, test, sub

# create batches of 4000 observations
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

def diff_features(df, shifts):
    for shift in shifts:    
        df['signal_diff_shift_' + str(shift)] = df.groupby('group')['signal'].diff(shift).fillna(0)
    return df

def categorize(df, thres):
    df = df > thres
    return df

def cat_features(df, thresholds):
    for thres in thresholds:    
        df['signal_cat_' + str(thres)] = df.groupby('group')['signal'].apply(lambda x: categorize(x, thres)).astype(float)
    return df

# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    df = diff_features(df, [-1, 1])
    df = cat_features(df, [-2, -1, 0, 1, 2, 3])
    # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal'] ** 2
    return df


# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

# model function (very important, you can try different arquitectures to get a better score. I believe that top public leaderboard is a 1D Conv + RNN style)
def Classifier(shape_):
    
    def cbr(x, out_layer, kernel, stride, dilation):
        x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
    
    inp = Input(shape = (shape_))
    x = cbr(inp, 64, 7, 1, 1)
    x = BatchNormalization()(x)
    x = wave_block(x, 16, 3, 12)
    x = BatchNormalization()(x)
    x = wave_block(x, 32, 3, 8)
    x = BatchNormalization()(x)
    x = wave_block(x, 64, 3, 4)
    x = BatchNormalization()(x)
    x = wave_block(x, 128, 3, 1)
    x = cbr(x, 32, 7, 1, 1)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    out = Dense(11, activation = 'softmax', name = 'out')(x)
    
    model = models.Model(inputs = inp, outputs = out)
    
    opt = Adam(lr = LR)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
    return model

# function that decrease the learning as epochs increase (i also change this part of the code)
def lr_schedule(epoch):
    if epoch < 30:
        lr = LR
    elif epoch < 40:
        lr = LR / 3
    elif epoch < 50:
        lr = LR / 5
    elif epoch < 60:
        lr = LR / 7
    elif epoch < 70:
        lr = LR / 9
    elif epoch < 80:
        lr = LR / 11
    elif epoch < 90:
        lr = LR / 13
    else:
        lr = LR / 100
    return lr

# class to get macro f1 score. This is not entirely necessary but it's fun to check f1 score of each epoch (be carefull, if you use this function early stopping callback will not work)
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis = 2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis = 2).reshape(-1)
        score = f1_score(self.targets, pred, average = 'macro')
        print(f'F1 Macro Score: {score:.5f}')

# main function to perfrom groupkfold cross validation (we have 1000 vectores of 4000 rows and 8 features (columns)). Going to make 5 groups with this subgroups.
def run_cv_model_by_batch(train, test, splits, batch_col, feats, sample_submission, nn_epochs, nn_batch_size):
    
    seed_everything(SEED)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
    tf.compat.v1.keras.backend.set_session(sess)
    oof_ = np.zeros((len(train), 11)) # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']
    kf = GroupKFold(n_splits=5)
    splits = [x for x in kf.split(train, train[target], group)]

    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])    
        new_splits.append(new_split)
    # pivot target columns to transform the net to a multiclass classification estructure (you can also leave it in 1 vector with sparsecategoricalcrossentropy loss function)
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]], axis=1)

    tr.columns = ['target_'+str(i) for i in range(11)] + ['group']
    target_cols = ['target_'+str(i) for i in range(11)]
    train_tr = np.array(list(tr.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(list(train.groupby('group').apply(lambda x: x[feats].values)))
    test = np.array(list(test.groupby('group').apply(lambda x: x[feats].values)))

    for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]
        print(f'Our training dataset shape is {train_x.shape}')
        print(f'Our validation dataset shape is {valid_x.shape}')

        gc.collect()
        shape_ = (None, train_x.shape[2]) # input is going to be the number of feature we are using (dimension 2 of 0, 1, 2)
        model = Classifier(shape_)
        # using our lr_schedule function
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        model.fit(train_x,train_y,
                  epochs = nn_epochs,
                  callbacks = [cb_lr_schedule, MacroF1(model, valid_x, valid_y)], # adding custom evaluation metric for each epoch
                  batch_size = nn_batch_size,verbose = 2,
                  validation_data = (valid_x,valid_y))
        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro') # need to get the class with the biggest probability
        print(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx,:] += preds_f
        te_preds = model.predict(test)
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
        preds_ += te_preds / SPLITS
    # calculate the oof macro f1_score
    f1_score_ = f1_score(np.argmax(train_tr, axis = 2).reshape(-1),  np.argmax(oof_, axis = 1), average = 'macro') # axis 2 for the 3 Dimension array and axis 1 for the 2 Domension Array (extracting the best class)
    print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
    sample_submission['open_channels'] = np.argmax(preds_, axis = 1).astype(int)
    sample_submission.to_csv('submission_wavenet_base_v54_noise.csv', index=False, float_format='%.4f')
    
    np.save('preds_base_v54_noise.npy', preds_)
    np.save('oof_base_v54_noise.npy', oof_)
    
# this function run our entire program
def run_everything():
    
    print('Reading Data Started...')
    train, test, sample_submission = read_data()
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')
        
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')
        
   
    print(f'Training Wavenet model with {SPLITS} folds of GroupKFold Started...')
    run_cv_model_by_batch(train, test, SPLITS, 'group', features, sample_submission, EPOCHS, NNBATCHSIZE)
    print('Training completed...')
        
run_everything()

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Feature Engineering Completed...
Training Wavenet model with 5 folds of GroupKFold Started...
Our training dataset shape is (1000, 4000, 28)
Our validation dataset shape is (250, 4000, 28)
Train on 1000 samples, validate on 250 samples
Epoch 1/224
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
F1 Macro Score: 0.64598
1000/1000 - 35s - loss: 0.6392 - accuracy: 0.8240 - val_loss: 1.2383 - val_accuracy: 0.7884
Epoch 2/224
F1 Macro Score: 0.65682
1000/1000 - 9s - loss: 0.2145 - accuracy: 0.9510 - val_loss: 0.7647 - val_accuracy: 0.8279
Epoch 3/224
F1 Macro Score: 0.92868
1000/1000 - 9s - loss: 0.1598 - accuracy: 0.9613 - val_loss: 0.2787 - val_accuracy: 0.9600
Epoch 4/224
F1 Macro Score: 0.93316
1000/1000 - 9s - loss: 0.1424 - accuracy: 0.9638 - val_loss: 0.1681 - val_accuracy: 0.9641
Epoch 5/224
F1 Macro Score: 0.93544
1000/1000 - 9s - loss: 0.1

Epoch 60/224
F1 Macro Score: 0.93823
1000/1000 - 9s - loss: 0.0870 - accuracy: 0.9685 - val_loss: 0.0874 - val_accuracy: 0.9670
Epoch 61/224
F1 Macro Score: 0.93829
1000/1000 - 9s - loss: 0.0868 - accuracy: 0.9684 - val_loss: 0.0876 - val_accuracy: 0.9670
Epoch 62/224
F1 Macro Score: 0.93824
1000/1000 - 8s - loss: 0.0869 - accuracy: 0.9684 - val_loss: 0.0869 - val_accuracy: 0.9670
Epoch 63/224
F1 Macro Score: 0.93754
1000/1000 - 8s - loss: 0.0865 - accuracy: 0.9685 - val_loss: 0.0886 - val_accuracy: 0.9667
Epoch 64/224
F1 Macro Score: 0.93844
1000/1000 - 8s - loss: 0.0866 - accuracy: 0.9685 - val_loss: 0.0871 - val_accuracy: 0.9670
Epoch 65/224
F1 Macro Score: 0.93839
1000/1000 - 8s - loss: 0.0863 - accuracy: 0.9686 - val_loss: 0.0869 - val_accuracy: 0.9671
Epoch 66/224
F1 Macro Score: 0.93823
1000/1000 - 8s - loss: 0.0864 - accuracy: 0.9684 - val_loss: 0.0871 - val_accuracy: 0.9670
Epoch 67/224
F1 Macro Score: 0.93843
1000/1000 - 8s - loss: 0.0865 - accuracy: 0.9685 - val_loss: 0.0869

Epoch 124/224
F1 Macro Score: 0.93924
1000/1000 - 8s - loss: 0.0830 - accuracy: 0.9693 - val_loss: 0.0853 - val_accuracy: 0.9675
Epoch 125/224
F1 Macro Score: 0.93922
1000/1000 - 8s - loss: 0.0833 - accuracy: 0.9693 - val_loss: 0.0854 - val_accuracy: 0.9675
Epoch 126/224
F1 Macro Score: 0.93931
1000/1000 - 8s - loss: 0.0833 - accuracy: 0.9693 - val_loss: 0.0853 - val_accuracy: 0.9676
Epoch 127/224
F1 Macro Score: 0.93913
1000/1000 - 8s - loss: 0.0849 - accuracy: 0.9688 - val_loss: 0.0855 - val_accuracy: 0.9675
Epoch 128/224
F1 Macro Score: 0.93917
1000/1000 - 8s - loss: 0.0832 - accuracy: 0.9693 - val_loss: 0.0854 - val_accuracy: 0.9675
Epoch 129/224
F1 Macro Score: 0.93928
1000/1000 - 8s - loss: 0.0831 - accuracy: 0.9693 - val_loss: 0.0853 - val_accuracy: 0.9675
Epoch 130/224
F1 Macro Score: 0.93929
1000/1000 - 8s - loss: 0.0828 - accuracy: 0.9695 - val_loss: 0.0853 - val_accuracy: 0.9675
Epoch 131/224
F1 Macro Score: 0.93925
1000/1000 - 8s - loss: 0.0833 - accuracy: 0.9693 - val_loss

Epoch 188/224
F1 Macro Score: 0.93932
1000/1000 - 8s - loss: 0.0824 - accuracy: 0.9695 - val_loss: 0.0851 - val_accuracy: 0.9675
Epoch 189/224
F1 Macro Score: 0.93944
1000/1000 - 8s - loss: 0.0825 - accuracy: 0.9695 - val_loss: 0.0852 - val_accuracy: 0.9676
Epoch 190/224
F1 Macro Score: 0.93923
1000/1000 - 8s - loss: 0.0825 - accuracy: 0.9695 - val_loss: 0.0852 - val_accuracy: 0.9675
Epoch 191/224
F1 Macro Score: 0.93938
1000/1000 - 8s - loss: 0.0822 - accuracy: 0.9695 - val_loss: 0.0850 - val_accuracy: 0.9676
Epoch 192/224
F1 Macro Score: 0.93948
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9696 - val_loss: 0.0851 - val_accuracy: 0.9676
Epoch 193/224
F1 Macro Score: 0.93934
1000/1000 - 8s - loss: 0.0823 - accuracy: 0.9695 - val_loss: 0.0851 - val_accuracy: 0.9676
Epoch 194/224
F1 Macro Score: 0.93942
1000/1000 - 8s - loss: 0.0822 - accuracy: 0.9696 - val_loss: 0.0851 - val_accuracy: 0.9676
Epoch 195/224
F1 Macro Score: 0.93941
1000/1000 - 8s - loss: 0.0820 - accuracy: 0.9696 - val_loss

Epoch 27/224
F1 Macro Score: 0.93897
1000/1000 - 8s - loss: 0.0975 - accuracy: 0.9666 - val_loss: 0.0848 - val_accuracy: 0.9696
Epoch 28/224
F1 Macro Score: 0.93738
1000/1000 - 8s - loss: 0.0973 - accuracy: 0.9665 - val_loss: 0.0907 - val_accuracy: 0.9685
Epoch 29/224
F1 Macro Score: 0.93908
1000/1000 - 8s - loss: 0.0980 - accuracy: 0.9665 - val_loss: 0.0862 - val_accuracy: 0.9694
Epoch 30/224
F1 Macro Score: 0.93868
1000/1000 - 8s - loss: 0.0968 - accuracy: 0.9666 - val_loss: 0.0856 - val_accuracy: 0.9694
Epoch 31/224
F1 Macro Score: 0.93887
1000/1000 - 8s - loss: 0.0943 - accuracy: 0.9669 - val_loss: 0.0839 - val_accuracy: 0.9695
Epoch 32/224
F1 Macro Score: 0.93953
1000/1000 - 8s - loss: 0.0932 - accuracy: 0.9670 - val_loss: 0.0832 - val_accuracy: 0.9697
Epoch 33/224
F1 Macro Score: 0.93995
1000/1000 - 8s - loss: 0.0931 - accuracy: 0.9671 - val_loss: 0.0818 - val_accuracy: 0.9700
Epoch 34/224
F1 Macro Score: 0.94013
1000/1000 - 8s - loss: 0.0923 - accuracy: 0.9673 - val_loss: 0.0816

...

Epoch 153/224
F1 Macro Score: 0.94040
1000/1000 - 8s - loss: 0.0816 - accuracy: 0.9698 - val_loss: 0.0813 - val_accuracy: 0.9692
Epoch 154/224
F1 Macro Score: 0.94041
1000/1000 - 8s - loss: 0.0817 - accuracy: 0.9697 - val_loss: 0.0813 - val_accuracy: 0.9692
Epoch 155/224
F1 Macro Score: 0.94036
1000/1000 - 8s - loss: 0.0821 - accuracy: 0.9697 - val_loss: 0.0813 - val_accuracy: 0.9691
Epoch 156/224
F1 Macro Score: 0.94037
1000/1000 - 8s - loss: 0.0821 - accuracy: 0.9696 - val_loss: 0.0813 - val_accuracy: 0.9691
Epoch 157/224
F1 Macro Score: 0.94030
1000/1000 - 8s - loss: 0.0816 - accuracy: 0.9698 - val_loss: 0.0813 - val_accuracy: 0.9691
Epoch 158/224
F1 Macro Score: 0.94033
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9697 - val_loss: 0.0813 - val_accuracy: 0.9691
Epoch 159/224
F1 Macro Score: 0.94037
1000/1000 - 8s - loss: 0.0816 - accuracy: 0.9698 - val_loss: 0.0813 - val_accuracy: 0.9692
Epoch 160/224
F1 Macro Score: 0.94033
1000/1000 - 8s - loss: 0.0816 - accuracy: 0.9698 - val_loss

Epoch 217/224
F1 Macro Score: 0.94037
1000/1000 - 8s - loss: 0.0809 - accuracy: 0.9700 - val_loss: 0.0811 - val_accuracy: 0.9692
Epoch 218/224
F1 Macro Score: 0.94045
1000/1000 - 8s - loss: 0.0813 - accuracy: 0.9699 - val_loss: 0.0811 - val_accuracy: 0.9692
Epoch 219/224
F1 Macro Score: 0.94043
1000/1000 - 8s - loss: 0.0812 - accuracy: 0.9699 - val_loss: 0.0812 - val_accuracy: 0.9692
Epoch 220/224
F1 Macro Score: 0.94049
1000/1000 - 8s - loss: 0.0814 - accuracy: 0.9699 - val_loss: 0.0810 - val_accuracy: 0.9692
Epoch 221/224
F1 Macro Score: 0.94037
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9697 - val_loss: 0.0811 - val_accuracy: 0.9692
Epoch 222/224
F1 Macro Score: 0.94041
1000/1000 - 8s - loss: 0.0808 - accuracy: 0.9700 - val_loss: 0.0811 - val_accuracy: 0.9692
Epoch 223/224
F1 Macro Score: 0.94035
1000/1000 - 8s - loss: 0.0812 - accuracy: 0.9698 - val_loss: 0.0811 - val_accuracy: 0.9692
Epoch 224/224
F1 Macro Score: 0.94047
1000/1000 - 8s - loss: 0.0811 - accuracy: 0.9699 - val_loss

Epoch 56/224
F1 Macro Score: 0.93918
1000/1000 - 8s - loss: 0.0871 - accuracy: 0.9684 - val_loss: 0.0835 - val_accuracy: 0.9685
Epoch 57/224
F1 Macro Score: 0.93908
1000/1000 - 8s - loss: 0.0862 - accuracy: 0.9685 - val_loss: 0.0831 - val_accuracy: 0.9686
Epoch 58/224
F1 Macro Score: 0.93897
1000/1000 - 8s - loss: 0.0856 - accuracy: 0.9687 - val_loss: 0.0837 - val_accuracy: 0.9685
Epoch 59/224
F1 Macro Score: 0.93899
1000/1000 - 8s - loss: 0.0862 - accuracy: 0.9686 - val_loss: 0.0833 - val_accuracy: 0.9685
Epoch 60/224
F1 Macro Score: 0.93897
1000/1000 - 8s - loss: 0.0863 - accuracy: 0.9687 - val_loss: 0.0840 - val_accuracy: 0.9684
Epoch 61/224
F1 Macro Score: 0.93907
1000/1000 - 8s - loss: 0.0858 - accuracy: 0.9687 - val_loss: 0.0829 - val_accuracy: 0.9685
Epoch 62/224
F1 Macro Score: 0.93898
1000/1000 - 8s - loss: 0.0856 - accuracy: 0.9687 - val_loss: 0.0828 - val_accuracy: 0.9686
Epoch 63/224
F1 Macro Score: 0.93849
1000/1000 - 8s - loss: 0.0853 - accuracy: 0.9687 - val_loss: 0.0837

Epoch 120/224
F1 Macro Score: 0.93954
1000/1000 - 8s - loss: 0.0824 - accuracy: 0.9695 - val_loss: 0.0818 - val_accuracy: 0.9688
Epoch 121/224
F1 Macro Score: 0.93944
1000/1000 - 8s - loss: 0.0827 - accuracy: 0.9694 - val_loss: 0.0818 - val_accuracy: 0.9688
Epoch 122/224
F1 Macro Score: 0.93952
1000/1000 - 8s - loss: 0.0830 - accuracy: 0.9693 - val_loss: 0.0819 - val_accuracy: 0.9688
Epoch 123/224
F1 Macro Score: 0.93956
1000/1000 - 8s - loss: 0.0825 - accuracy: 0.9694 - val_loss: 0.0818 - val_accuracy: 0.9689
Epoch 124/224
F1 Macro Score: 0.93960
1000/1000 - 8s - loss: 0.0823 - accuracy: 0.9694 - val_loss: 0.0818 - val_accuracy: 0.9688
Epoch 125/224
F1 Macro Score: 0.93950
1000/1000 - 8s - loss: 0.0825 - accuracy: 0.9694 - val_loss: 0.0818 - val_accuracy: 0.9688
Epoch 126/224
F1 Macro Score: 0.93959
1000/1000 - 8s - loss: 0.0830 - accuracy: 0.9693 - val_loss: 0.0818 - val_accuracy: 0.9688
Epoch 127/224
F1 Macro Score: 0.93952
1000/1000 - 8s - loss: 0.0837 - accuracy: 0.9692 - val_loss

Epoch 184/224
F1 Macro Score: 0.93964
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9696 - val_loss: 0.0817 - val_accuracy: 0.9688
Epoch 185/224
F1 Macro Score: 0.93957
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9696 - val_loss: 0.0817 - val_accuracy: 0.9689
Epoch 186/224
F1 Macro Score: 0.93956
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9695 - val_loss: 0.0817 - val_accuracy: 0.9688
Epoch 187/224
F1 Macro Score: 0.93952
1000/1000 - 8s - loss: 0.0818 - accuracy: 0.9696 - val_loss: 0.0817 - val_accuracy: 0.9688
Epoch 188/224
F1 Macro Score: 0.93949
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9696 - val_loss: 0.0817 - val_accuracy: 0.9688
Epoch 189/224
F1 Macro Score: 0.93945
1000/1000 - 8s - loss: 0.0820 - accuracy: 0.9695 - val_loss: 0.0817 - val_accuracy: 0.9689
Epoch 190/224
F1 Macro Score: 0.93931
1000/1000 - 8s - loss: 0.0820 - accuracy: 0.9696 - val_loss: 0.0819 - val_accuracy: 0.9688
Epoch 191/224
F1 Macro Score: 0.93962
1000/1000 - 8s - loss: 0.0819 - accuracy: 0.9695 - val_loss